In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
from PIL import Image # 用於圖像縮放，如果你沒有安裝PIL/Pillow，請先安裝：pip install Pillow

def generate_pixelated_apple(output_size=(64, 64), filename="low_res_apple.mat"):
    """
    生成一個畫質很差的蘋果圖片，並保存為 .mat 檔案。

    Args:
        output_size (tuple): 輸出圖片的尺寸 (寬, 高)。
        filename (str): 保存的 .mat 檔案名稱。
    """
    # 1. 生成一個簡單的蘋果形狀數據 (較高解析度，以便降採樣)
    # 我們可以畫一個圓形作為蘋果的主體，然後在頂部加上一個小方形作為梗。
    img_res = 256 # 初始較高解析度
    Y, X = np.ogrid[0:img_res, 0:img_res]

    # 蘋果主體 (稍微偏下方的圓形)
    center_x, center_y = img_res / 2, img_res * 0.55
    radius = img_res * 0.3
    apple_body = ((X - center_x)**2 + (Y - center_y)**2 <= radius**2).astype(float)

    # 蘋果梗 (頂部的小長方形)
    stem_width = img_res * 0.05
    stem_height = img_res * 0.1
    stem_x_start = int(center_x - stem_width / 2)
    stem_x_end = int(center_x + stem_width / 2)
    stem_y_start = int(center_y - radius - stem_height)
    stem_y_end = int(center_y - radius)

    # 確保索引不越界
    stem_y_start = max(0, stem_y_start)
    stem_y_end = min(img_res, stem_y_end)
    stem_x_start = max(0, stem_x_start)
    stem_x_end = min(img_res, stem_x_end)

    if stem_y_end > stem_y_start and stem_x_end > stem_x_start:
        apple_body[stem_y_start:stem_y_end, stem_x_start:stem_x_end] = 1.0


    # 2. 應用 Jet Colormap
    # 我們將 apple_body 數據映射到 0-255 範圍，以便 colormap 處理
    # 然後通過 matplotlib 的 colormap 獲取 RGBA 顏色數據
    norm_apple_body = apple_body * 255
    cmap = plt.get_cmap('jet')
    # cmap 函數返回的是 RGBA 浮點數，範圍 0-1
    color_mapped_image = cmap(norm_apple_body.astype(int) / 255.0)

    # color_mapped_image 現在是 (img_res, img_res, 4) 的 numpy 數組 (RGBA)
    # 我們只需要 RGB，所以取前三個通道
    rgb_image_float = color_mapped_image[:, :, :3]

    # 將浮點數轉換為 0-255 整數
    rgb_image_int = (rgb_image_float * 255).astype(np.uint8)

    # 3. 降採樣 (降低畫質)
    # 將 NumPy 陣列轉換為 PIL 圖像物件進行縮放，這通常效果更好
    pil_img = Image.fromarray(rgb_image_int)
    # 使用最近鄰插值 (Image.NEAREST) 來實現像素化效果
    low_res_pil_img = pil_img.resize(output_size, Image.NEAREST)

    # 轉換回 NumPy 陣列
    low_res_np_img = np.array(low_res_pil_img)

    # 4. 保存為 .mat 檔案
    # MATLAB 的影像數據通常是 (高, 寬, 通道) 或 (高, 寬)
    # 這裡我們保存 RGB 數據
    mdic = {"apple_image": low_res_np_img}
    savemat(filename, mdic)

    print(f"低畫質蘋果圖片已生成並保存為 '{filename}'")

    # (可選) 顯示圖片以確認效果
    plt.imshow(low_res_np_img)
    plt.title(f"Low Resolution Apple ({output_size[0]}x{output_size[1]})")
    plt.axis('off')
    plt.show()

# 執行函數生成圖片
generate_pixelated_apple(output_size=(64, 64), filename="low_res_apple.mat")

: 